In [1]:
#Import important Libraries
import numpy as np
import pandas as pd
%pylab inline
import seaborn as sns
import matplotlib.pyplot as plt
import re
# Importing Neccessary Libraries
import os

Populating the interactive namespace from numpy and matplotlib


In [2]:
path = os.getcwd()

In [3]:
df_ajah_rent = pd.read_csv(path + '/Areas/ajah_rent_clean.csv')
df_gbagada_rent = pd.read_csv(path + '/Areas/gbagada_rent_clean.csv')
df_ikeja_rent = pd.read_csv(path + '/Areas/ikeja_rent_clean.csv')
df_ikorodu_rent = pd.read_csv(path + '/Areas/ikorodu_rent_clean.csv')
df_ikoyi_rent = pd.read_csv(path + '/Areas/ikoyi_rent_clean.csv')
df_lekki_rent = pd.read_csv(path + '/Areas/lekki_rent_clean.csv')
df_surulere_rent = pd.read_csv(path + '/Areas/surulere_rent_clean.csv')
df_yaba_rent = pd.read_csv(path + '/Areas/yaba_rent_clean.csv')


In [4]:
df_rent = pd.concat([df_ajah_rent, df_gbagada_rent, df_ikeja_rent, df_ikorodu_rent, df_ikoyi_rent, df_lekki_rent, df_surulere_rent, df_yaba_rent], join='outer', ignore_index=True)

In [6]:
df_rent.drop('Unnamed: 0', 1, inplace=True)
df_rent.head(3)

,Property,Location,Description,Features,Price,Bed,Bath,Toilet,Built or Serviced,Property Type,Area,Purchase Type,Parking Space,Security,Electricity,Furnished,Security Doors,CCTV
0,2 bedroom flat / apartment for rent,...sangotedo ajah lagos,a very spacious and sweet 2 bedroom flat en-su...,2 bedrooms,800000.0,2.0,3.0,3.0,NaN,Flat / apartment,ajah,rent,0,0,0,0,0,0
1,1 bedroom mini flat boys quarters flat / apart...,.sangotedo ajah lagos,a very sweet minni flat for rent very close to...,1 bedrooms,700000.0,1.0,2.0,2.0,NaN,Mini flat,ajah,rent,0,0,0,0,0,0
2,1 bedroom mini flat mini flat flat / apartment...,lekki palm city ajah lagosajah lagos,a newly built 1 bedroom mini flat with visitor...,1 bedrooms\nnew,900000.0,1.0,1.0,2.0,Newly Built,Mini flat,ajah,rent,0,0,0,0,0,0


In [7]:
         ### CREATE NEW COLUMNS FOR HOUSES ATRIBUTES THAT MAY AFFECT THE PRICE ###
## BOY QUATERS ##
df_rent['BQ'] = np.where(((df_rent.Description.str.contains('bq'))|(df_rent.Features.str.contains('bq'))), 1, 0)

## GYM ##
df_rent['Gym'] = np.where(((df_rent.Description.str.contains('gym'))|(df_rent.Features.str.contains('gym'))), 1, 0)

## POOL ##
df_rent['Pool'] = np.where(((df_rent.Description.str.contains('pool'))|(df_rent.Features.str.contains('pool'))), 1, 0)
                


# BASIC EDA

In [8]:
print("The number of rows and colums are {} and also called shape of the matrix".format(df_rent.shape))


The number of rows and colums are (30622, 21) and also called shape of the matrix


In [9]:
print("Columns names are \n {}".format(df_rent.columns))

Columns names are 
 Index(['Property', 'Location', 'Description', 'Features', 'Price', 'Bed',
       'Bath', 'Toilet', 'Built or Serviced', 'Property Type', 'Area',
       'Purchase Type', 'Parking Space', 'Security', 'Electricity',
       'Furnished', 'Security Doors', 'CCTV', 'BQ', 'Gym', 'Pool'],
      dtype='object')


In [10]:
df_rent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30622 entries, 0 to 30621
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Property           30622 non-null  object 
 1   Location           30622 non-null  object 
 2   Description        30616 non-null  object 
 3   Features           29091 non-null  object 
 4   Price              30622 non-null  float64
 5   Bed                29902 non-null  float64
 6   Bath               29622 non-null  float64
 7   Toilet             28906 non-null  float64
 8   Built or Serviced  9145 non-null   object 
 9   Property Type      30622 non-null  object 
 10  Area               30622 non-null  object 
 11  Purchase Type      30622 non-null  object 
 12  Parking Space      30622 non-null  int64  
 13  Security           30622 non-null  int64  
 14  Electricity        30622 non-null  int64  
 15  Furnished          30622 non-null  int64  
 16  Security Doors     306

In [11]:
pd.crosstab(df_rent['Bed'],df_rent['Area'],values=df_rent['Price'], aggfunc='count' )

Area,ajah,gbagada,ikeja,ikorodu,ikoyi,lekki,surulere,yaba
Bed,,,,,,,,
0.0,203.0,78.0,100.0,26.0,62.0,220.0,120.0,351.0
1.0,1426.0,835.0,699.0,228.0,156.0,526.0,1197.0,2166.0
2.0,1479.0,1156.0,626.0,509.0,408.0,362.0,1185.0,1648.0
3.0,1940.0,1689.0,1652.0,359.0,1910.0,692.0,995.0,1510.0
4.0,306.0,107.0,395.0,13.0,885.0,876.0,68.0,45.0
5.0,27.0,15.0,128.0,1.0,194.0,240.0,3.0,6.0
6.0,6.0,1.0,15.0,NaN,17.0,6.0,NaN,NaN
7.0,NaN,NaN,5.0,NaN,3.0,4.0,NaN,NaN
8.0,NaN,NaN,3.0,NaN,3.0,NaN,NaN,NaN


In [13]:
# Before working on houses with bedrooms equal to zero
df_rent[((df_rent['Bed']==0)|(df_rent['Bed'].isna())) & (df_rent['Description'].str.contains('bedroom'))].shape

(500, 21)

In [14]:
# Getting information of number of bedrooms from description for houses with beds equal to zero or nan.
d1 =df_rent[((df_rent['Bed']==0)|(df_rent['Bed'].isna())) & (df_rent['Description'].str.contains('bedroom'))]

In [16]:
# Before inserting values for beds, baths and rooms
d1.head(2)

,Property,Location,Description,Features,Price,Bed,Bath,Toilet,Built or Serviced,Property Type,...,Purchase Type,Parking Space,Security,Electricity,Furnished,Security Doors,CCTV,BQ,Gym,Pool
108,flat / apartment for rent,ajah lagos,2bedroom for let at mobile road ajah,NaN,900000.0,NaN,NaN,NaN,NaN,Flat / apartment,...,rent,0,0,0,0,0,0,0,0,0
133,flat / apartment for rent,terra anex estatesangotedo ajah lagos,to let 2bedroom flat at terra anex estate sheg...,NaN,800000.0,0.0,0.0,0.0,NaN,Flat / apartment,...,rent,0,0,0,0,0,0,0,0,0


In [17]:
# List of numbers of room gotten from description information
mylist = [int(re.findall('\d+',(re.findall('\d+bedroom', x.replace(' ','')))[0])[0]) if len(re.findall('\d+bedroom', x.replace(' ',''))) > 0 else 0
        for x in d1['Description']]

In [18]:
# Inserting values gotten from the description
d1.loc[:, 'Bed']= mylist
d1.loc[:, 'Bath']= mylist
d1.loc[:, 'Toilet']=[y+1 if y!= 0 else y for y in mylist]

C:\Users\HP\Anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [19]:
# After inserting values for beds, baths and rooms
d1.head(2)

,Property,Location,Description,Features,Price,Bed,Bath,Toilet,Built or Serviced,Property Type,...,Purchase Type,Parking Space,Security,Electricity,Furnished,Security Doors,CCTV,BQ,Gym,Pool
108,flat / apartment for rent,ajah lagos,2bedroom for let at mobile road ajah,NaN,900000.0,2,2,3,NaN,Flat / apartment,...,rent,0,0,0,0,0,0,0,0,0
133,flat / apartment for rent,terra anex estatesangotedo ajah lagos,to let 2bedroom flat at terra anex estate sheg...,NaN,800000.0,2,2,3,NaN,Flat / apartment,...,rent,0,0,0,0,0,0,0,0,0


In [20]:
#Assigning these values to the original Dataframe
df_rent[((df_rent['Bed']==0)|(df_rent['Bed'].isna()))  & (df_rent['Description'].str.contains('bedroom'))] = d1

In [21]:
# Assigning bath value to houses with bedroom == 0 or Nan. An intelligent guess.
df_rent[((df_rent['Bed']==0)|(df_rent['Bed'].isnull())) & (df_rent['Bath']==1)] = df_rent[((df_rent['Bed']==0)|(df_rent['Bed'].isnull())) & (df_rent['Bath']==1)].assign(Bed = 1, inplace=True)
df_rent[((df_rent['Bed']==0)|(df_rent['Bed'].isnull())) & (df_rent['Bath']==2)] = df_rent[((df_rent['Bed']==0)|(df_rent['Bed'].isnull())) & (df_rent['Bath']==2)].assign(Bed = 2, inplace=True)
df_rent[((df_rent['Bed']==0)|(df_rent['Bed'].isnull())) & (df_rent['Bath']==3)] = df_rent[((df_rent['Bed']==0)|(df_rent['Bed'].isnull())) & (df_rent['Bath']==3)].assign(Bed = 3, inplace=True)
df_rent[(df_rent['Bed'].isnull()) & (df_rent['Property Type'] == 'Self contain')] = df_rent[(df_rent['Bed'].isnull()) & (df_rent['Property Type'] == 'Self contain')].assign(Bed=1, Bath=1, Toilet=1, inplace=True)

In [22]:
# Dropping rows with bed, bath and toilet value equal to zero
df_rent.drop(df_rent[(df_rent['Bed']==0)].index, inplace = True) 

In [102]:
# Dropping Self Contain houses with bedrooms greater than 1
df_rent.drop(df_rent[(df_rent['Property Type'] == 'Self contain') & (df_rent['Bed'] > 1)].index, inplace=True)

In [24]:
df_rent[df_rent['Price'] < 200000]['Area'].unique()

array(['ajah', 'gbagada', 'ikeja', 'ikorodu', 'ikoyi', 'lekki',
       'surulere', 'yaba'], dtype=object)

In [ ]:
# EDA on Prices

In [131]:
# Price values in this filter are not plausible so its assumed to be false and ought to be dropped.
df_rent.drop(df_rent[(df_rent['Price'] <= 400000) & (df_rent['Area'] == 'ikoyi')].index, inplace=True)
df_rent.drop(df_rent[(df_rent['Price'] <= 200000) & (df_rent['Area'] == 'lekki')].index, inplace=True) 
df_rent.drop(df_rent[(df_rent['Price'] <= 500000) & (df_rent['Area'] == 'lekki') & (df_rent['Bed'] > 2)].index, inplace=True)

In [25]:
# Dropping properties which are short lets
df_rent.drop(df_rent[(df_rent['Description'].str.contains('shortlet')) & (pd.notna(df_rent['Description']))].index, inplace=True)

In [111]:
pd.DataFrame({'Percentage_Mising':(df_rent.isnull().sum()/df_rent.shape[0])* 100})

,Percentage_Mising
Property,0.000000
Location,0.000000
Description,0.021007
Features,3.091520
Price,0.000000
Bed,0.000000
Bath,0.000000
Toilet,0.000000
Property Type,0.000000
Area,0.000000


In [27]:
# The rent Built or Serviced column has more than 50% missing data, so it should be dropped.
df_rent.drop('Built or Serviced', 1, inplace=True)

In [92]:
# Filling Bed, Bath and Toilet nan values with 1 for mini flats being mini flats have 1 room.
tmp = df_rent[(df_rent['Bed'].isnull()) & (df_rent['Description'].str.contains('mini flat')|df_rent['Description'].str.contains('self contain'))]
tmp[['Bed','Bath','Toilet']] = tmp[['Bed','Bath','Toilet']].fillna(1)
df_rent[(df_rent['Bed'].isnull()) & (df_rent['Description'].str.contains('mini flat')|df_rent['Description'].str.contains('self contain'))] = tmp

C:\Users\HP\Anaconda3\lib\site-packages\pandas\core\frame.py:3062: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [74]:
# Filling Bed, Bath and Toilet nan values with 2 and 1 based on the condition in mylist2 for flats in Ajah.
mylist2 = np.where(tmp['Price'] >500000, 2, 1).tolist()
tmp = df_rent[(df_rent['Bed'].isnull()) & (df_rent['Area'] == 'ajah')]
tmp['Bed'] = mylist2
tmp['Bath'] = mylist2
tmp['Toilet'] = mylist2
df_rent[(df_rent['Bed'].isnull()) & (df_rent['Area'] == 'ajah')] = tmp


In [79]:
# Filling Bed, Bath and Toilet nan values with 1 based on the assumption housing less than 400000 in price are mostly 1 bedrooms
tmp = df_rent[(df_rent['Bed'].isnull()) & (df_rent['Price'] < 400000)]
tmp[['Bed','Bath','Toilet']] = tmp[['Bed','Bath','Toilet']].fillna(1)
df_rent[(df_rent['Bed'].isnull()) & (df_rent['Price'] < 400000)] = tmp

C:\Users\HP\Anaconda3\lib\site-packages\pandas\core\frame.py:3062: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [90]:
# Filling Bed, Bath and Toilet nan values with 1 based on the assumption Mini flats are 1 bedroom apartments fro Property Type Mini flat
tmp = df_rent[(df_rent['Bed'].isnull()) & (df_rent['Property Type']=='Mini flat')]
tmp[['Bed','Bath','Toilet']] = tmp[['Bed','Bath','Toilet']].fillna(1)
df_rent[(df_rent['Bed'].isnull()) & (df_rent['Property Type']=='Mini flat')] = tmp   


C:\Users\HP\Anaconda3\lib\site-packages\pandas\core\frame.py:3062: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [98]:
# Filling Bed, Bath and Toilet nan values with 3 on the condition 3 is forund in the description column
tmp = df_rent[(df_rent['Bed'].isnull()) & (df_rent['Description'].str.contains('3'))]
tmp[['Bed','Bath']] = tmp[['Bed','Bath']].fillna(3)
tmp['Toilet'] = tmp['Toilet'].fillna(4)
df_rent[(df_rent['Bed'].isnull()) & (df_rent['Description'].str.contains('3'))] = tmp

In [106]:
# Dropping the rest rows with missing bed, bath and toilet values
df_rent.dropna(subset=['Bed', 'Bath', 'Toilet'], inplace=True)

In [109]:
cleaned_data = df_rent.drop(['Property','Location','Description','Features',],axis=1)

In [110]:
cleaned_data.head()

,Price,Bed,Bath,Toilet,Property Type,Area,Purchase Type,Parking Space,Security,Electricity,Furnished,Security Doors,CCTV,BQ,Gym,Pool
0,800000.0,2.0,3.0,3.0,Flat / apartment,ajah,rent,0,0,0,0,0,0,0,0,0
1,700000.0,1.0,2.0,2.0,Mini flat,ajah,rent,0,0,0,0,0,0,0,0,0
2,900000.0,1.0,1.0,2.0,Mini flat,ajah,rent,0,0,0,0,0,0,0,0,0
3,1100000.0,3.0,3.0,4.0,Flat / apartment,ajah,rent,0,0,0,0,0,0,0,0,0
4,1800000.0,4.0,4.0,4.0,Detached bungalow,ajah,rent,1,0,0,1,0,0,1,0,0


In [130]:
# Saving cleaned data to local disk
cleaned_data.to_csv(path+'/cleaned_rent.csv',index=False)